In [2]:
%pip install mlflow

  Using cached docker-7.1.0-py3-none-any.whl.metadata (3.8 kB)
  Using cached graphene-3.4.3-py2.py3-none-any.whl.metadata (6.9 kB)
  Using cached gunicorn-23.0.0-py3-none-any.whl.metadata (4.4 kB)
  Using cached fastapi-0.116.1-py3-none-any.whl.metadata (28 kB)
  Using cached opentelemetry_api-1.36.0-py3-none-any.whl.metadata (1.5 kB)
  Using cached opentelemetry_sdk-1.36.0-py3-none-any.whl.metadata (1.5 kB)
  Using cached sqlparse-0.5.3-py3-none-any.whl.metadata (3.9 kB)
  Using cached uvicorn-0.35.0-py3-none-any.whl.metadata (6.5 kB)
  Using cached mako-1.3.10-py3-none-any.whl.metadata (2.9 kB)
  Using cached itsdangerous-2.2.0-py3-none-any.whl.metadata (1.9 kB)
  Using cached werkzeug-3.1.3-py3-none-any.whl.metadata (3.7 kB)
  Using cached graphql_core-3.2.6-py3-none-any.whl.metadata (11 kB)
  Using cached graphql_relay-3.2.0-py3-none-any.whl.metadata (12 kB)
  Using cached google_auth-2.40.3-py2.py3-none-any.whl.metadata (6.2 kB)
  Using cached starlette-0.47.3-py3-none-any.whl.me

In [14]:
%pip install dagshub

  Using cached dagshub-0.6.3-py3-none-any.whl.metadata (12 kB)
  Using cached dacite-1.6.0-py3-none-any.whl.metadata (14 kB)
  Using cached gql-4.0.0-py3-none-any.whl.metadata (10 kB)
  Using cached treelib-1.8.0-py3-none-any.whl.metadata (3.3 kB)
  Using cached pathvalidate-3.3.1-py3-none-any.whl.metadata (12 kB)
  Using cached jmespath-1.0.1-py3-none-any.whl.metadata (7.6 kB)
  Using cached s3transfer-0.13.1-py3-none-any.whl.metadata (1.7 kB)
  Using cached backoff-2.2.1-py3-none-any.whl.metadata (14 kB)
  Using cached requests_toolbelt-1.0.0-py2.py3-none-any.whl.metadata (14 kB)
Using cached dagshub-0.6.3-py3-none-any.whl (261 kB)
Using cached dacite-1.6.0-py3-none-any.whl (12 kB)
Using cached pathvalidate-3.3.1-py3-none-any.whl (24 kB)
Using cached treelib-1.8.0-py3-none-any.whl (30 kB)
Using cached backoff-2.2.1-py3-none-any.whl (15 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.0/14.0 MB 63.2 MB/s eta 0:00:0000:01
Using cached jmespath-1.0.1-py3-none-any.whl (20 kB)
Using cac

In [21]:
import mlflow
import pandas as pd
import os 
import mlflow.sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd
import re
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import numpy as np
import dagshub
from dotenv import load_dotenv
load_dotenv()  

MLFLOW_TRACKING_URI = os.getenv("MLFLOW_TRACKING_URI")
DAGSHUB_REPO_OWNER = os.getenv("DAGSHUB_REPO_OWNER")
DAGSHUB_REPO_NAME = os.getenv("DAGSHUB_REPO_NAME")

In [4]:
df = pd.read_csv('IMDB.csv')
df = df.sample(500)
df.to_csv('data.csv', index=False)
df.head()

,review,sentiment
871,Legend has it that at the gala Hollywood premi...,negative
473,"Okay, you have:<br /><br />Penelope Keith as M...",negative
729,This is one horror movie based TV show that ge...,positive
49,A wonder. One of the best musicals ever. The t...,positive
65,"Most movies I can sit through easily, even if ...",negative


In [5]:
# data preprocessing

# Define text preprocessing functions
def lemmatization(text):
    """Lemmatize the text."""
    lemmatizer = WordNetLemmatizer()
    text = text.split()
    text = [lemmatizer.lemmatize(word) for word in text]
    return " ".join(text)

def remove_stop_words(text):
    """Remove stop words from the text."""
    stop_words = set(stopwords.words("english"))
    text = [word for word in str(text).split() if word not in stop_words]
    return " ".join(text)

def removing_numbers(text):
    """Remove numbers from the text."""
    text = ''.join([char for char in text if not char.isdigit()])
    return text

def lower_case(text):
    """Convert text to lower case."""
    text = text.split()
    text = [word.lower() for word in text]
    return " ".join(text)

def removing_punctuations(text):
    """Remove punctuations from the text."""
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = text.replace('؛', "")
    text = re.sub('\s+', ' ', text).strip()
    return text

def removing_urls(text):
    """Remove URLs from the text."""
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

def normalize_text(df):
    """Normalize the text data."""
    try:
        df['review'] = df['review'].apply(lower_case)
        df['review'] = df['review'].apply(remove_stop_words)
        df['review'] = df['review'].apply(removing_numbers)
        df['review'] = df['review'].apply(removing_punctuations)
        df['review'] = df['review'].apply(removing_urls)
        df['review'] = df['review'].apply(lemmatization)
        return df
    except Exception as e:
        print(f'Error during text normalization: {e}')
        raise

<>:32: SyntaxWarning: invalid escape sequence '\s'
<>:32: SyntaxWarning: invalid escape sequence '\s'
/var/folders/rm/ndzwygt56bl2x6plgskms_sm0000gn/T/ipykernel_10629/3798096103.py:32: SyntaxWarning: invalid escape sequence '\s'
  text = re.sub('\s+', ' ', text).strip()


In [6]:
df = normalize_text(df)
df.head()

,review,sentiment
871,legend gala hollywood premiere screening space...,negative
473,okay have br br penelope keith miss herringbon...,negative
729,one horror movie based tv show get right frida...,positive
49,wonder one best musical ever three busby berke...,positive
65,movie sit easily even particularly like movie ...,negative


In [7]:
df['sentiment'].value_counts()

sentiment
negative    265
positive    235
Name: count, dtype: int64

In [8]:
x = df['sentiment'].isin(['positive','negative'])
df = df[x]

In [9]:
df['sentiment'] = df['sentiment'].map({'positive':1, 'negative':0})
df.head()

,review,sentiment
871,legend gala hollywood premiere screening space...,0
473,okay have br br penelope keith miss herringbon...,0
729,one horror movie based tv show get right frida...,1
49,wonder one best musical ever three busby berke...,1
65,movie sit easily even particularly like movie ...,0


In [10]:
df.isnull().sum()

review       0
sentiment    0
dtype: int64

In [11]:
vectorizer = CountVectorizer(max_features=100)
X = vectorizer.fit_transform(df['review'])
y = df['sentiment']

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [15]:
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)
dagshub.init(repo_owner=DAGSHUB_REPO_OWNER, repo_name= DAGSHUB_REPO_NAME, mlflow=True)

# mlflow.set_experiment("Logistic Regression Baseline")
mlflow.set_experiment("Logistic Regression Baseline")


❗❗❗ AUTHORIZATION REQUIRED ❗❗❗

Output()



Open the following link in your browser to authorize the client:
https://dagshub.com/login/oauth/authorize?state=df11cd73-4d91-42d8-a59b-c235ee98cb11&client_id=32b60ba385aa7cecf24046d8195a71c07dd345d9657977863b52e7748e0f0f28&middleman_request_id=ce00965598f1aa25f2dd6d3adf4e2bee01cffeb65342d61cceab6623319155c4




Accessing as GauriNaik826

Repository MLops doesn't exist, creating it under current user.

Initialized MLflow to track repo "GauriNaik826/MLops"

Repository GauriNaik826/MLops initialized!

2025/08/27 15:49:26 INFO mlflow.tracking.fluent: Experiment with name 'Logistic Regression Baseline' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/e0eaf58056f3472c9d92add63f7a50f6', creation_time=1756334966895, experiment_id='0', last_update_time=1756334966895, lifecycle_stage='active', name='Logistic Regression Baseline', tags={}>

In [19]:
import mlflow
import logging
import mlflow.sklearn
import os
import time
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Configure logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

logging.info("Starting MLflow run...")

with mlflow.start_run():
    start_time = time.time()
    
    try:
        logging.info("Logging preprocessing parameters...")
        mlflow.log_param("vectorizer", "Bag of Words")
        mlflow.log_param("num_features", 100)
        mlflow.log_param("test_size", 0.25)

        logging.info("Initializing Logistic Regression model...")
        model = LogisticRegression(max_iter=1000)  # Increase max_iter to prevent non-convergence issues

        logging.info("Fitting the model...")
        model.fit(X_train, y_train)
        logging.info("Model training complete.")

        logging.info("Logging model parameters...")
        mlflow.log_param("model", "Logistic Regression")

        logging.info("Making predictions...")
        y_pred = model.predict(X_test)

        logging.info("Calculating evaluation metrics...")
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)

        logging.info("Logging evaluation metrics...")
        mlflow.log_metric("accuracy", accuracy)
        mlflow.log_metric("precision", precision)
        mlflow.log_metric("recall", recall)
        mlflow.log_metric("f1_score", f1)

        logging.info("Saving and logging the model...")
        mlflow.sklearn.save_model(model, "sk_model")        # writes to a local dir
        mlflow.log_artifacts("sk_model", artifact_path="model")  # uploads that folder

        # Log execution time
        end_time = time.time()
        logging.info(f"Model training and logging completed in {end_time - start_time:.2f} seconds.")

        # Save and log the notebook
        # notebook_path = "exp1_baseline_model.ipynb"
        # logging.info("Executing Jupyter Notebook. This may take a while...")
        # os.system(f"jupyter nbconvert --to notebook --execute --inplace {notebook_path}")
        # mlflow.log_artifact(notebook_path)

        # logging.info("Notebook execution and logging complete.")

        # Print the results for verification
        logging.info(f"Accuracy: {accuracy}")
        logging.info(f"Precision: {precision}")
        logging.info(f"Recall: {recall}")
        logging.info(f"F1 Score: {f1}")

    except Exception as e:
        logging.error(f"An error occurred: {e}", exc_info=True)


2025-08-27 16:04:33,118 - INFO - Starting MLflow run...
2025-08-27 16:04:33,306 - INFO - Logging preprocessing parameters...
2025-08-27 16:04:33,942 - INFO - Initializing Logistic Regression model...
2025-08-27 16:04:33,942 - INFO - Fitting the model...
2025-08-27 16:04:33,954 - INFO - Model training complete.
2025-08-27 16:04:33,954 - INFO - Logging model parameters...
2025-08-27 16:04:34,273 - INFO - Making predictions...
2025-08-27 16:04:34,273 - INFO - Calculating evaluation metrics...
2025-08-27 16:04:34,277 - INFO - Logging evaluation metrics...
2025-08-27 16:04:39,394 - INFO - Saving and logging the model...
2025-08-27 16:04:45,480 - INFO - Model training and logging completed in 12.17 seconds.
2025-08-27 16:04:45,483 - INFO - Accuracy: 0.632
2025-08-27 16:04:45,484 - INFO - Precision: 0.6862745098039216
2025-08-27 16:04:45,484 - INFO - Recall: 0.5384615384615384
2025-08-27 16:04:45,485 - INFO - F1 Score: 0.603448275862069


🏃 View run luxuriant-conch-786 at: https://dagshub.com/GauriNaik826/MLops_end_to_end.mlflow/#/experiments/0/runs/8c12fba436184ffeb436a36c13aa6d66
🧪 View experiment at: https://dagshub.com/GauriNaik826/MLops_end_to_end.mlflow/#/experiments/0
